# 链家租房数据爬虫与分析

本项目包含从链家网爬取租房数据、数据清洗、可视化分析以及价格预测建模的完整流程。

## 1. 环境准备与配置

In [ ]:
import logging
import random
import re
import time
from datetime import datetime, timedelta
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
from bs4 import BeautifulSoup, Tag
from catboost import CatBoostRegressor, Pool
from fake_useragent import UserAgent
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(message)s',
    datefmt='%H:%M:%S'
)

logger = logging.getLogger(__name__)

In [ ]:
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / "data"
IMAGE_DIR = PROJECT_ROOT / "images"

DATA_DIR.mkdir(
    parents=True,
    exist_ok=True
)

IMAGE_DIR.mkdir(
    parents=True,
    exist_ok=True
)

COOKIE = """lianjia_ssid=4a28a7ab-95c3-4a2a-adb2-614ada8aa564; lianjia_uuid=70e937bd-096c-43b1-876d-c72b58e8a945; crosSdkDT2019DeviceId=g4b33x-e5a4d0-07iwd5eo9m7trtn-38ptqziht; hip=AqseMDeEUD8KnaJz31REUlqEVoAaBOj7ZhKOvpCFlfRS3TlDE-eWH8vGpJgrXmHrygkJVi3BtewtwaW_BNc0KBRz5uE325UkFQNawYoIgGV1vg_GyrX8I8h8yBGOfSD9LUMuCSSYkGx93AvLwasdws2L1DDytuw1VxLjcaGmg0hXDaYQCi9xGxfDHMjhtA6SSsxU9nTKm4ERmnaX7N7PWzh7d5-ZSDsDH5EGOq4XdnElORe-kbYhWEMTq4g%3D; select_city=350200; GUARANTEE_POPUP_SHOW=true; login_ucid=2000000512872949; lianjia_token=2.0013adb9bb48a505a20200908a7396f6a0; lianjia_token_secure=2.0013adb9bb48a505a20200908a7396f6a0; security_ticket=nYOKAMd9n/QpLtgFkorqsClo/Elzf7O8IHDZLiibijk5gnMta1g4DwgwUytslHSsb+Wz1EqewjXZ3po58cIZ0E7LclTtf26iM5PlXgKCKoLr8s6t1sj2LPgvDDhfs5RcTYl5WIQSixY86e/AA2Y6n6Bf4PcQsuq3RCCz6h69MnM=; ftkrc_=a9c712e6-2d9e-44ff-a8a7-8b676f62ef8b; lfrc_=16737aa3-77b2-4858-8626-97a1824f2268; srcid=eyJ0Ijoie1wiZGF0YVwiOlwiODY5ZWMwMWNiMzgwNDUzMTVjNDAwMTQwODU2MjJkODk3NjQyYTVjYTIzYjE2NTE2ZDA1N2UwZTk3ZDRjYTZmMzlhOWJmZDVhZjNlMjY5MTE3OTZlY2Q3YjFjZTYwMzk5MmM4MDM2NzQyNDVmYmYzNGNiZTQ2MGQyMjk2YmE5OWI4NTg3Y2JmMDVhOWU1MmYyNmY0ZWI1NzZmMWY2OGVkZmU1NGM1MGQ5YjJkN2E0ZmZlZmEzNzJjZGIzNWM4M2U0OTVhMDVlNzBmMmM3YjczNDhhNzU2NTc1YmViMmY3ZTAwNzhlMmNmNzc2ZGEwZjc1MWJkYjJjZmMzNzdlYjhmNlwiLFwia2V5X2lkXCI6XCIxXCIsXCJzaWduXCI6XCI4MTAzODhmMFwifSIsInIiOiJodHRwczovL3htLmxpYW5qaWEuY29tL3p1ZmFuZy8iLCJvcyI6IndlYiIsInYiOiIwLjEifQ==; GUARANTEE_BANNER_SHOW=true"""

START_PAGE = 1
END_PAGE = 50
MIN_DELAY = 6
MAX_DELAY = 8
MAX_FAILURES = 3

CITIES_MAP = {
    'bj': '北京',
    'sh': '上海',
    'gz': '广州',
    'sz': '深圳',
    'wh': '武汉',
    'cd': '成都',
    'hz': '杭州',
    'xm': '厦门',
    'fz': '福州'
}

ANALYSIS_CITY = '上海'

## 2. 工具类定义

In [ ]:
class CsvUtil:
    @staticmethod
    def save_data(
        dataset: Union[pd.DataFrame, List[Dict]],
        perfix: str,
        identifier: Optional[str] = None
    ) -> Path:
        df = dataset if isinstance(dataset, pd.DataFrame) else pd.DataFrame(dataset)

        file_stem = f"{perfix}_{identifier}" if identifier else perfix
        output_path = DATA_DIR / f"{file_stem}.csv"

        df.to_csv(output_path, index=False, encoding='utf-8-sig')

        return output_path

    @staticmethod
    def load_data(perfix: str, match_multiple: bool = False) -> Optional[pd.DataFrame]:
        search_pattern = f"{perfix}_*.csv" if match_multiple else f"{perfix}.csv"
        found_files = list(DATA_DIR.glob(search_pattern))

        if not found_files:
            return None

        return pd.concat(
            [pd.read_csv(file_path, encoding='utf-8-sig') for file_path in found_files],
            ignore_index=True
        )

class ImageUtil:
    @staticmethod
    def save_plot(
        figure: go.Figure,
        file_name: str,
        image_format: str = 'html',
        scale_factor: float = 2.0
    ) -> str:
        output_path = IMAGE_DIR / f"{file_name}.{image_format}"

        if image_format == 'html':
            figure.write_html(output_path)
        else:
            figure.write_image(output_path, scale=scale_factor)

        return str(output_path)

## 3. 阶段一：数据爬取

In [ ]:
class LianJiaSpider:
    _PATTERNS = {
        'layout': re.compile(r'(\d+)室(\d+)厅(\d+)卫'),
        'area': re.compile(r'(\d+\.?\d*)㎡'),
        'orientation': re.compile(r'/\s*([\u4e00-\u9fa5\s]+)\s*/'),
        'floor_level': re.compile(r'([\u4e00-\u9fa5]+)楼层'),
        'total_floors': re.compile(r'（(\d+)层）')
    }

    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': UserAgent().random,
            'Cookie': COOKIE
        })

    def run(
            self,
            specific_cities: Dict[str, str]
    ):
        for city_code, city_name in specific_cities.items():
            data = self._crawl_city(
                city_name,
                city_code
            )
            CsvUtil.save_data(
                data,
                prefix='raw',
                city_name=city_name
            )

    def _crawl_city(
            self,
            city_name: str,
            city_code: str
    ) -> List[Dict]:
        logger.info(f">>> 启动抓取: {city_name} ({city_code})")
        items = []
        consecutive_failures = 0

        for page in range(START_PAGE, END_PAGE + 1):
            page_data = self._fetch_page(
                city_code,
                page,
                city_name
            )

            if page_data:
                items.extend(page_data)
                consecutive_failures = 0
                logger.info(f"    √ 已抓取 {len(items)} 条数据 (当前页: {page})")
            else:
                consecutive_failures += 1
                logger.info(f"    × 第 {page} 页无数据")

            if consecutive_failures >= MAX_FAILURES:
                logger.error(f"连续失败触发熔断: {city_name}")
                break

            time.sleep(random.uniform(MIN_DELAY, MAX_DELAY))

        logger.info(f"<<< {city_name} 抓取完成，共 {len(items)} 条")
        return items

    def _fetch_page(
            self,
            city_code: str,
            page: int,
            city_name: str
    ) -> List[Dict]:
        url = f"https://{city_code}.lianjia.com/zufang/pg{page}/"
        try:
            resp = self.session.get(
                url,
                timeout=10
            )
            resp.raise_for_status()
            soup = BeautifulSoup(
                resp.content,
                'html.parser'
            )
            cards = soup.select('div.content__list--item')
            return [
                self._parse_card(card, city_name) for card in cards
            ]
        except Exception as e:
            logger.error(f"请求异常 (Page {page}): {e}")
            return []

    def _parse_card(
            self,
            card: Tag,
            city_name: str
    ) -> Dict[str, Any]:
        """解析单个房源卡片的 HTML 元素"""

        def _text(selector: str) -> str:
            el = card.select_one(selector)
            return el.get_text(strip=True) if el else ""

        def _regex(
                text: str,
                key: str,
                dtype: type = str,
                default: Any = None
        ) -> Any:
            # 使用预定义正则提取数据
            match = self._PATTERNS[key].search(text)
            if match:
                try:
                    if match.lastindex == 1:
                        return dtype(match.group(1))
                    return match.groups()
                except (ValueError, IndexError):
                    pass
            return default

        # 提取基础文本信息
        desc = _text('p.content__list--item--des')
        floor_txt = _text('p.content__list--item--des span.hide')
        title = _text('p.content__list--item--title')

        # 提取位置信息 (行政区, 商圈, 小区)
        locs = [
            a.get_text(strip=True) for a in card.select('p.content__list--item--des a')[:3]
        ]
        locs += [''] * (3 - len(locs))

        # 解析户型 (室/厅/卫)
        layout = self._PATTERNS['layout'].search(desc)
        rooms = [int(x) for x in layout.groups()] if layout else [0, 0, 0]

        tag_texts = [
            i.get_text(strip=True) for i in card.select('p.content__list--item--bottom i')
        ]

        return {
            'city': city_name,
            'title': title,
            'rent_type': '整租' if '整租' in title else ('合租' if '合租' in title else '独栋'),
            'district': locs[0],
            'sub_district': locs[1],
            'community': locs[2],
            'area_sqm': _regex(desc, 'area', float, 0.0),
            'bedrooms': rooms[0],
            'living_rooms': rooms[1],
            'bathrooms': rooms[2],
            'orientation': _regex(desc, 'orientation', str, '').strip(),
            'floor_level': _regex(floor_txt, 'floor_level', str, ''),
            'total_floors': _regex(floor_txt, 'total_floors', int, 0),
            'tags': '|'.join(tag_texts),
            'platform': _text('p.content__list--item--brand span.brand'),
            'update_time': _text('p.content__list--item--brand span.content__list--item--time'),
            'price_rmb': _text('span.content__list--item-price em')
        }

In [ ]:
# 执行爬虫
def get_missing_cities() -> dict:
    existing_files = list(DATA_DIR.glob("raw_*.csv"))
    existing_names = {
        p.stem.split('_')[1] for p in existing_files
    }
    return {
        c: n for c, n in CITIES_MAP.items() if n not in existing_names
    }

missing_cities = get_missing_cities()
if missing_cities:
    print(f"需要爬取的城市: {list(missing_cities.values())}")
    LianJiaSpider().run(missing_cities)
else:
    print("所有目标城市数据已存在，跳过爬取。")

## 4. 阶段二：数据清洗

In [ ]:
class DataCleaner:
    COLS_TEXT = [
        'city',
        'title',
        'rent_type',
        'district',
        'sub_district',
        'community',
        'orientation',
        'floor_level',
        'tags',
        'platform',
        'update_time',
        'price_rmb'
    ]
    COLS_NUM = [
        'area_sqm',
        'bedrooms',
        'living_rooms',
        'bathrooms',
        'total_floors'
    ]
    _PATTERN_PRICE = re.compile(r'(\d+\.?\d*)')
    _PATTERN_REL_DATE = re.compile(r'(\d+)\s*(天|周|个月|月|年)前')
    _DATE_OFFSET_MAP = {
        '天': 1,
        '周': 7,
        '月': 30,
        '年': 365
    }

    def __init__(
            self,
            df: pd.DataFrame
    ):
        self.df = df.copy()
        self.now = datetime.now()

    @classmethod
    def execute_task(cls) -> pd.DataFrame:
        raw_df = CsvUtil.load_data(
            "raw",
            match_multiple=True
        )
        if raw_df is None or raw_df.empty:
            logger.warning("无原始数据")
            return pd.DataFrame()

        cleaned_df = cls(raw_df).process()
        CsvUtil.save_data(
            cleaned_df,
            prefix='cleaned'
        )
        return cleaned_df

    def process(self) -> pd.DataFrame:
        initial_len = len(self.df)
        self.df = (
            self.df
            .pipe(self._clean_text)
            .pipe(self._clean_numeric)
            .pipe(self._parse_prices)
            .pipe(self._standardize_dates)
            .pipe(self._remove_outliers)
        )
        logger.info(f"清洗完成: {initial_len} -> {len(self.df)} (剔除 {initial_len - len(self.df)} 条)")
        return self.df

    def _clean_text(
            self,
            df: pd.DataFrame
    ) -> pd.DataFrame:
        cols = df.columns.intersection(self.COLS_TEXT)
        df[cols] = df[cols].fillna('未知').astype(str).apply(lambda x: x.str.strip())
        df[cols] = df[cols].replace(
            r'^(nan|NaN|None|NULL|)$',
            '未知',
            regex=True
        )
        return df

    def _clean_numeric(self, df: pd.DataFrame) -> pd.DataFrame:
        for col in df.columns.intersection(self.COLS_NUM):
            series = pd.to_numeric(df[col], errors='coerce')

            df[col] = series.fillna(0)
        return df

    def _parse_prices(
            self,
            df: pd.DataFrame
    ) -> pd.DataFrame:
        def _calc_stats(val: str) -> pd.Series:
            # 从字符串提取所有数字 (处理如 '3000-4000' 的区间价格)
            nums = [
                float(x) for x in self._PATTERN_PRICE.findall(str(val))
            ]
            if nums:
                return pd.Series([
                    min(nums),
                    max(nums),
                    sum(nums) / len(nums)  # 计算均价
                ])
            return pd.Series([0.0, 0.0, 0.0])

        stats = df['price_rmb'].apply(_calc_stats)
        stats.columns = [
            'price_min',
            'price_max',
            'price_avg'
        ]
        return pd.concat(
            [df, stats],
            axis=1
        )

    def _standardize_dates(
            self,
            df: pd.DataFrame
    ) -> pd.DataFrame:
        def _parse(date_str: str) -> str:
            if '今天' in date_str:
                return self.now.strftime('%Y-%m-%d')
            match = self._PATTERN_REL_DATE.search(date_str)
            if match:
                val, unit = match.groups()
                days = int(val) * self._DATE_OFFSET_MAP.get(unit.replace('个', ''), 0)
                return (self.now - timedelta(days=days)).strftime('%Y-%m-%d')
            return date_str

        df['clean_date'] = df['update_time'].apply(_parse)
        return df

    def _remove_outliers(
            self,
            df: pd.DataFrame
    ) -> pd.DataFrame:
        df = df[df['price_avg'] > 0]
        if df.empty:
            return df
        low, high = df['price_avg'].quantile(0.03), df['price_avg'].quantile(0.97)
        return df[(df['price_avg'] >= low) & (df['price_avg'] <= high)]

In [ ]:
df_clean = DataCleaner.execute_task()

if not df_clean.empty:
    display(df_clean.head())
    print(f"Data Shape: {df_clean.shape}")
else:
    print("没有数据可供清洗。")

## 5. 阶段三：数据可视化

In [ ]:
class RentalDataVisualizer:
    LABELS_MAP = {
        'city': '城市',
        'price_avg': '月租均价 (元)',
        'price_per_sqm': '每平米单价 (元/㎡)',
        'is_subway': '房源类型',
        'district': '行政区',
        'count': '房源数量',
        'range': '价格区间'
    }
    COLOR_SEQ = px.colors.qualitative.Plotly

    def __init__(
            self,
            df: pd.DataFrame
    ):
        self.df = df[df['district'] != '未知'].copy()
        self.df['price_per_sqm'] = self.df['price_avg'] / self.df['area_sqm']
        self.df['is_subway'] = np.where(
            self.df['tags'].str.contains('近地铁'),
            '地铁房',
            '普通房'
        )

    def _apply_style(
            self,
            fig,
            title: str
    ):
        fig.update_layout(
            title={
                'text': title,
                'x': 0.5,
                'y': 0.95,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            template="plotly_white",
            font=dict(
                family="Microsoft YaHei",
                size=12
            ),
            margin=dict(
                l=40,
                r=40,
                t=80,
                b=40
            ),
            showlegend=True
        )
        return fig

    def plot_city_comparison(self):
        metrics = self.df.groupby('city')[['price_avg', 'price_per_sqm']].mean().reset_index()
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        fig.add_trace(
            go.Bar(
                x=metrics['city'],
                y=metrics['price_avg'],
                name="月租均价",
                marker_color=self.COLOR_SEQ[0]
            ),
            secondary_y=False
        )

        fig.add_trace(
            go.Scatter(
                x=metrics['city'],
                y=metrics['price_per_sqm'],
                name="每平米单价",
                mode='lines+markers',
                line=dict(
                    color=self.COLOR_SEQ[1],
                    width=3
                )
            ),
            secondary_y=True
        )

        fig.update_yaxes(
            title_text="月租均价 (元)",
            secondary_y=False
        )
        fig.update_yaxes(
            title_text="每平米单价 (元/㎡)",
            secondary_y=True,
            showgrid=False
        )
        self._apply_style(
            fig,
            "各城市租金水平对比"
        )
        ImageUtil.save_plot(
            fig,
            "各城市租金水平对比"
        )
        return fig

    def plot_subway_premium(self):
        metrics = self.df.groupby('is_subway')['price_avg'].mean().reset_index()
        fig = px.bar(
            metrics,
            x='is_subway',
            y='price_avg',
            labels=self.LABELS_MAP,
            color='is_subway',
            color_discrete_sequence=self.COLOR_SEQ,
            text_auto='.2f'
        )
        fig.update_traces(textposition='outside')
        self._apply_style(
            fig,
            "地铁房 vs 普通房：月租均价对比"
        )
        ImageUtil.save_plot(
            fig,
            "地铁房 vs 普通房：月租均价对比"
        )
        return fig

    def plot_city_analysis(
            self,
            city: str
    ):
        city_data = self.df[self.df['city'] == city]
        if city_data.empty:
            logger.warning(f"未找到城市 {city} 的数据")
            return None

        # 1. Box Plot
        fig_box = px.box(
            city_data,
            x='district',
            y='price_avg',
            labels=self.LABELS_MAP,
            color='district',
            color_discrete_sequence=self.COLOR_SEQ
        )
        fig_box.update_xaxes(categoryorder='median ascending')
        self._apply_style(
            fig_box,
            f"{city} - 各区域租金分布"
        )
        ImageUtil.save_plot(
            fig_box,
            f"{city} - 各区域租金分布"
        )
        fig_box.show()

        # 2. Top 10
        top10 = city_data.groupby('district')['price_avg'].mean().nlargest(10).reset_index()
        fig_bar = px.bar(
            top10,
            x='price_avg',
            y='district',
            orientation='h',
            labels=self.LABELS_MAP,
            color='price_avg',
            color_continuous_scale='Blues'
        )
        fig_bar.update_yaxes(autorange="reversed")
        self._apply_style(
            fig_bar,
            f"{city} - 租金最贵行政区 Top 10"
        )
        ImageUtil.save_plot(
            fig_bar,
            f"{city} - 租金最贵行政区 Top 10"
        )
        fig_bar.show()

        # 3. Pie Chart
        bins = [0, 2000, 4000, 6000, 8000, 10000, float('inf')]
        labels = ['2k以下', '2k-4k', '4k-6k', '6k-8k', '8k-1w', '1w以上']
        counts = pd.cut(
            city_data['price_avg'],
            bins=bins,
            labels=labels
        ).value_counts().reset_index()
        counts.columns = ['range', 'count']

        fig_pie = px.pie(
            counts,
            values='count',
            names='range',
            labels=self.LABELS_MAP,
            color_discrete_sequence=self.COLOR_SEQ,
            hole=0.4
        )
        fig_pie.update_traces(
            textposition='inside',
            textinfo='percent+label'
        )
        self._apply_style(
            fig_pie,
            f"{city} - 房源价格区间占比"
        )
        ImageUtil.save_plot(
            fig_pie,
            f"{city} - 房源价格区间占比"
        )
        fig_pie.show()

In [ ]:
# 执行可视化
if not df_clean.empty:
    viz = RentalDataVisualizer(df_clean)

    # 1. 城市对比
    fig1 = viz.plot_city_comparison()
    fig1.show()

    # 2. 地铁房溢价
    fig2 = viz.plot_subway_premium()
    fig2.show()

    # 3. 特定城市深度分析
    viz.plot_city_analysis(ANALYSIS_CITY)

## 6. 阶段四：价格预测建模

In [ ]:
class ModelPipeline:
    CATEGORICAL_FEATURES = [
        'city',
        'rent_type',
        'district',
        'sub_district',
        'orientation',
        'floor_level'
    ]
    NUMERIC_FEATURES = [
        'area_sqm',
        'bedrooms',
        'living_rooms',
        'bathrooms',
        'total_floors',
        'is_subway'
    ]
    TARGET_COLUMN = 'price_avg'
    FEATURE_DISPLAY_MAP = {
        'city': '城市',
        'rent_type': '租赁方式',
        'district': '行政区',
        'sub_district': '商圈',
        'orientation': '朝向',
        'floor_level': '楼层等级',
        'area_sqm': '面积(㎡)',
        'bedrooms': '室数',
        'living_rooms': '厅数',
        'bathrooms': '卫数',
        'total_floors': '总楼层',
        'is_subway': '是否近地铁'
    }
    COLOR_SEQ = px.colors.qualitative.Plotly

    def __init__(
            self,
            raw_df: pd.DataFrame
    ):
        self.raw_df = raw_df
        self.performance_metrics: List[Dict] = []

    def run(self):
        features, target = self._prepare_data()
        self.all_feature_names = features.columns.tolist()

        train_x, test_x, train_y, test_y = train_test_split(
            features,
            target,
            test_size=0.2,
            random_state=42
        )

        print("Training CatBoost...")
        catboost_model = self._train_catboost(
            train_x,
            train_y,
            test_x,
            test_y
        )
        self._evaluate(
            catboost_model,
            test_x,
            test_y,
            "CatBoost"
        )

        print("Training RandomForest...")
        rf_model, test_x_enc = self._train_random_forest(
            train_x,
            train_y,
            test_x
        )
        self._evaluate(
            rf_model,
            test_x_enc,
            test_y,
            "RandomForest"
        )

        self._plot_metrics_comparison()
        self._plot_importance(catboost_model)

    def _apply_style(self, fig, title: str):
        """
        统一图表样式配置：
        - 标题居中
        - 白色背景
        - 微软雅黑字体
        - 统一边距
        """
        fig.update_layout(
            title={'text': title, 'x': 0.5, 'y': 0.95, 'xanchor': 'center', 'yanchor': 'top'},
            template="plotly_white",
            font=dict(family="Microsoft YaHei", size=12),
            margin=dict(l=40, r=40, t=80, b=40),
            showlegend=True
        )
        return fig

    def _prepare_data(self) -> Tuple[pd.DataFrame, pd.Series]:
        df = self.raw_df.copy()
        # 构造 '是否近地铁' 特征 (0/1)
        df['is_subway'] = df['tags'].str.contains(
            '近地铁',
            na=False
        ).astype(int)

        return (
            df[self.CATEGORICAL_FEATURES + self.NUMERIC_FEATURES],
            df[self.TARGET_COLUMN]
        )

    def _train_catboost(
            self,
            train_x,
            train_y,
            test_x,
            test_y
    ):
        model = CatBoostRegressor(
            iterations=1000,
            learning_rate=0.05,
            depth=5,
            loss_function='RMSE',
            verbose=0,
            early_stopping_rounds=50,
            allow_writing_files=False
        )
        model.fit(
            Pool(
                train_x,
                train_y,
                cat_features=self.CATEGORICAL_FEATURES
            ),
            eval_set=Pool(
                test_x,
                test_y,
                cat_features=self.CATEGORICAL_FEATURES
            ),
            use_best_model=True
        )
        return model

    def _train_random_forest(
            self,
            train_x,
            train_y,
            test_x
    ):
        encoder = OrdinalEncoder(
            handle_unknown='use_encoded_value',
            unknown_value=-1
        )
        train_x_enc = train_x.copy()
        test_x_enc = test_x.copy()

        train_x_enc[self.CATEGORICAL_FEATURES] = encoder.fit_transform(
            train_x[self.CATEGORICAL_FEATURES]
        )
        test_x_enc[self.CATEGORICAL_FEATURES] = encoder.transform(
            test_x[self.CATEGORICAL_FEATURES]
        )

        model = RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            n_jobs=-1,
            random_state=42
        )
        model.fit(
            train_x_enc,
            train_y
        )
        return model, test_x_enc

    def _evaluate(
            self,
            model,
            features,
            target,
            model_name: str
    ):
        pred = model.predict(features)
        rmse = np.sqrt(mean_squared_error(target, pred))
        mae = mean_absolute_error(target, pred)
        r2 = r2_score(target, pred)

        print(f"{model_name}: RMSE={rmse:.2f}, MAE={mae:.2f}, R2={r2:.3f}")

        self.performance_metrics.extend([
            {'Model': model_name, 'Metric': 'RMSE', 'Value': rmse},
            {'Model': model_name, 'Metric': 'MAE', 'Value': mae},
            {'Model': model_name, 'Metric': 'R2', 'Value': r2}
        ])

    def _plot_metrics_comparison(self):
        metrics_df = pd.DataFrame(self.performance_metrics)
        fig = make_subplots(
            rows=1,
            cols=2,
            subplot_titles=("模型拟合度 (R2)", "误差指标 (RMSE/MAE)")
        )

        r2_df = metrics_df[metrics_df['Metric'] == 'R2']
        fig.add_trace(
            go.Bar(
                x=r2_df['Model'],
                y=r2_df['Value'],
                name='R2',
                text=r2_df['Value'].round(3),
                textposition='auto',
                marker_color=self.COLOR_SEQ[0]
            ),
            row=1,
            col=1
        )

        err_df = metrics_df[metrics_df['Metric'].isin(['RMSE', 'MAE'])]
        for i, metric in enumerate(['RMSE', 'MAE']):
            d = err_df[err_df['Metric'] == metric]
            fig.add_trace(
                go.Bar(
                    x=d['Model'],
                    y=d['Value'],
                    name=metric,
                    marker_color=self.COLOR_SEQ[i + 1]
                ),
                row=1,
                col=2
            )

        fig = self._apply_style(fig, "模型性能评估对比")
        fig.update_layout(barmode='group')

        ImageUtil.save_plot(
            fig,
            "模型性能对比图"
        )
        fig.show()

    def _plot_importance(
            self,
            model
    ):
        importance_df = pd.DataFrame({
            'feature': self.all_feature_names,
            'score': model.get_feature_importance()
        })
        importance_df['feature_cn'] = importance_df['feature'].map(self.FEATURE_DISPLAY_MAP)
        importance_df = importance_df.sort_values(
            'score',
            ascending=True
        ).tail(10)

        fig = px.bar(
            importance_df,
            x='score',
            y='feature_cn',
            orientation='h',
            labels={'score': '贡献度', 'feature_cn': '特征'},
            color='score',
            color_continuous_scale='Viridis'
        )

        # [修改点] 应用统一样式
        fig = self._apply_style(fig, "CatBoost 核心特征贡献排行")

        ImageUtil.save_plot(
            fig,
            "CatBoost 核心特征贡献排行"
        )
        fig.show()

In [ ]:
if not df_clean.empty:
    ModelPipeline(df_clean).run()
else:
    print("没有数据可供建模。")